<a href="https://colab.research.google.com/github/sinahuss/solar-flare-prediction/blob/main/notebooks/solar_flare_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# C964 Capstone: Solar Flare Prediction and Analysis


## 1. Business Understanding

### Organizational Need

Space weather events, particularly solar flares, pose significant risks to critical infrastructure on Earth and in space. Organizations like NOAA's Space Weather Prediction Center require reliable early warning systems to protect:

- Satellite communications and GPS systems
- Power grids
- Astronauts and aircraft
- Radio communications

Current prediction methods rely heavily on human expertise and limited historical patterns, which may result in missed events or false alarms. These risks can lead to potentially billions of dollars in economic damage and disruptions to essential services.

### Project Goal

This project aims to develop a data product featuring a machine learning model that can predict the likelihood of solar flare events (C, M, or X class) within a 24-hour period based on characteristics of sunspot regions. The model will provide early warning capability for space weather forecasters, and improved accuracy in flare prediction to reduce false alarms and missed events.

### Success Criteria

The model's success will be measured by:

- High recall for M and X class flares (the most dangerous events) to minimize missed warnings
- Balanced precision and recall to reduce false alarms while maintaining sensitivity
- Practical deployment feasibility for integration into existing space weather monitoring systems

This predictive capability would enable space weather agencies to provide more reliable warnings, allowing for better preparation and protection of critical infrastructure.


## 2. Data Understanding


### 2.1. Load Libraries and Data

Our solar flare prediction analysis begins with importing essential libraries and loading the sunspot dataset.

The dataset will be loaded from a public GitHub repository containing the Solar Flare Dataset from Kaggle, which provides the historical data needed to train our flare prediction model.

This dataset contains morphological characteristics of sunspot groups that solar physicists use to assess flare potential. The first few rows will be displayed to verify successful data loading and provide an initial glimpse of the sunspot characteristics.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import plotly.graph_objects as go
import plotly.express as px

# Load dataset from GitHub repository (public Kaggle dataset)
url = "https://raw.githubusercontent.com/sinahuss/solar-flare-prediction/refs/heads/main/data/data.csv"
df = pd.read_csv(url)

# Display first few rows to verify successful loading
df.head()

### 2.2. Dataset Feature Descriptions

The dataset contains 13 features describing each solar active region. The first 10 are the input features for our model, and the last three are the target variables we aim to predict.

**Input Features:**

- **modified Zurich class:** A classification of the sunspot group's magnetic complexity, generally ordered from least to most complex (A, B, C, D, E, F, H).
- **largest spot size:** Size of the largest spot in the group, ordered from smallest to largest (X, R, S, A, H, K).
- **spot distribution:** Compactness of the sunspot group, ordered from least to most compact (X, O, I, C).
- **activity:** A code representing the region's recent growth (1=decay, 2=no change).
- **evolution:** Describes the region's evolution over the last 24 hours (1=decay, 2=no growth, 3=growth).
- **previous 24 hour flare activity:** A code summarizing prior flare activity (1=none, 2=one M1, 3=>one M1).
- **historically-complex:** A flag indicating if the region was ever historically complex (1=Yes, 2=No).
- **became complex on this pass:** A flag indicating if the region became complex on its current transit (1=Yes, 2=No).
- **area:** A code for the total area of the sunspot group (1=small, 2=large).
- **area of largest spot:** A code for the area of the largest individual spot (1=<=5, 2=>5).

**Target Variables:**

- **common flares:** The number of C-class flares produced in the next 24 hours.
- **moderate flares:** The number of M-class flares produced in the next 24 hours.
- **severe flares:** The number of X-class flares produced in the next 24 hours.


### 2.3. Initial Data Inspection

A foundational understanding of the dataset's structure and quality must be established. This inspection is critical for the solar flare prediction model because data quality directly impacts model performance and reliability for space weather forecasting.

First, we will use `.info()` to examine the column names, data types, and check for any missing values. The output confirms that there are no missing values, meaning that null values do not have to be accounted for in the data preparation phase.

Next, we use `describe()` to generate a summary of the categorical features, including their unique values and most frequent entries, which helps us understand the distribution and composition of the dataset's categorical variables.


In [ ]:
df.info()

df.astype("object").describe().transpose()

### 2.4. Exploratory Data Analysis


#### 2.4.1. Target Variable Analysis

Before analyzing the input features, we must first understand the distribution of our target variables: `common flares`, `moderate flares`, and `severe flares`. The plots below show the number of 24-hour periods in the dataset that recorded zero, one, two, or more flares of each type.

**Key Findings:**

The visualization reveals a severe class imbalance for our solar flare prediction. Out of all 24-hour periods available, only 15% experienced at least one C-Class event, 5% recorded M-Class events, and 1% showed X-Class events.

This imbalance has several implications for our machine learning approach:

1. **Model Selection:** Traditional accuracy metrics will be misleading due to the dominance of the "no flare" class, so there should be higher focus on precision, recall, and F1-score.

2. **Sampling Strategy:** We may need to employ techniques like stratified sampling to address the imbalance.

3. **Evaluation Metrics:** The model's success will be measured primarily by its ability to correctly identify the rare but dangerous M and X-class flares, rather than overall accuracy.

4. **Business Impact:** Missing an X-class flare (false negative) is far more costly than incorrectly predicting one (false positive), making recall for severe flares our primary optimization target.


In [ ]:
flare_columns = ["common flares", "moderate flares", "severe flares"]

# Create a figure with 3 subplots, one for each flare type
fig, axes = plt.subplots(1, 3, figsize=(16, 5), sharey=True)
fig.suptitle("Distribution of Raw Flare Counts Per 24-Hour Period")

# Loop through each flare type and plot its distribution
for i, col in enumerate(flare_columns):
    ax = axes[i]
    countplot = sns.countplot(
        data=df, x=col, ax=ax, hue=col, palette="viridis", legend=False
    )
    ax.set_title(f"Distribution of {col}")
    ax.set_xlabel("Flares Recorded")
    for container in ax.containers:
        ax.bar_label(container, fmt="%d", label_type="edge", padding=2)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

#### 2.4.2. Relationship Analysis

We now explore the relationship between flare production and modified Zurich class through two visualizations. These analyses investigate a key hypothesis: that more complex sunspot groups produce more significant flares.

**Total Flares Analysis:** The first subplot shows the total number of C, M, and X-class flares produced by each modified Zurich class, revealing which sunspot configurations are the most prolific sources of solar flares.

**Average Flares Analysis:** The second subplot normalizes this data by showing the average number of flares per class instance, accounting for the different frequencies of each modified Zurich class in the dataset. This provides a more accurate assessment of flare risk per sunspot group.

**Key Findings:**

The two visualizations help us prioritize which modified Zurich class to observe.

- **Low-Risk:** B and C class sunspot regions are low complexity and produce the least amount of solar flares, so they can be seen as low-risk regions. H class regions are decayed remnants of C, D, E, and F regions, and are also low-risk regions.

- **Medium-Risk:** D class sunspot regions are interesting because they produce the highest number of total solar flares in the dataset. But, after normalizing the data, we can see that they actually produce significantly fewer flares per sunspot region. Therefore, they can be categorized as medium-risk regions.

- **High-Risk:** E class regions are almost guaranteed to produce solar flares, reaching just under 1 C-Class solar flare per instance. F class regions produce a low total amount of solar flares, but adjusting for their lower representation in the dataset, they produce a high number of solar flares per region. F class regions also produce the highest amount of X-class (severe) flares when data is normalized.


In [ ]:
# Melt the dataframe to have a single column for flare type and another for the count
flare_counts_df = df.melt(
    id_vars=["modified Zurich class"],
    value_vars=["common flares", "moderate flares", "severe flares"],
    var_name="flare_type",
    value_name="count",
)

# Specify the order for each categorical feature for consistent plotting
category_orders = {
    "modified Zurich class": ["B", "C", "D", "E", "F", "H"],
    "largest spot size": ["X", "R", "S", "A", "H", "K"],
    "spot distribution": ["X", "O", "I", "C"],
}

# Remove rows where flares have not occurred
flare_counts_df = flare_counts_df[flare_counts_df["count"] > 0]

# Calculate the number of sunspot groups for each Zurich class
zurich_class_counts = df["modified Zurich class"].value_counts().to_dict()

# Calculate the proportional number of flares (per Zurich class instance)
flare_counts_df["class_count"] = flare_counts_df["modified Zurich class"].map(
    zurich_class_counts
)
flare_counts_df["count_per_class"] = (
    flare_counts_df["count"] / flare_counts_df["class_count"]
)

# Create a figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Create the Grouped Bar Plot
sns.barplot(
    data=flare_counts_df,
    x="modified Zurich class",
    y="count",
    hue="flare_type",
    estimator=sum,
    order=category_orders["modified Zurich class"],
    palette="viridis",
    errorbar=None,
    ax=ax1,
)
ax1.set_title("Total Flares Produced by Sunspot Zurich Class")
ax1.set_xlabel("Modified Zurich Class")
ax1.set_ylabel("Total Number of Flares Recorded")
ax1.legend(title="Flare Type")

# Second subplot: Average Flares per Class Instance
sns.barplot(
    data=flare_counts_df,
    x="modified Zurich class",
    y="count_per_class",
    hue="flare_type",
    estimator=sum,
    order=category_orders["modified Zurich class"],
    palette="viridis",
    errorbar=None,
    ax=ax2,
)
ax2.set_title("Average Number of Flares per Sunspot Zurich Class")
ax2.set_xlabel("Modified Zurich Class")
ax2.set_ylabel("Average Number of Flares per Class Instance")
ax2.legend(title="Flare Type")

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

## 3. Data Preparation


### 3.1. Feature Engineering

The dataset tracks C, M, and X class flares in three separate columns, representing the count of each event type. For this classification task, a single target variable is needed. A new column will be created called `flare_class` that categorizes each sunspot region by the most significant flare it has produced in the following 24-hour period. The values 0, 1, 2, and 3 correspond to 'None', 'C', 'M', and 'X' class flares, respectively.

The original flare columns are dropped to prevent data leakage. This step ensures that the model will be trained on features that are predictive rather than features that contain information about the target variable itself.

In [ ]:
# Determine the highest flare class for each row
def get_flare_class(row):
    if row["severe flares"] > 0:
        return 3  # X-class
    elif row["moderate flares"] > 0:
        return 2  # M-class
    elif row["common flares"] > 0:
        return 1  # C-class
    else:
        return 0  # None


# Create a new target column
df["flare_class"] = df.apply(get_flare_class, axis=1)

print(df["flare_class"].value_counts())

# Drop original flare columns to prevent data leakage
df.drop(columns=["common flares", "moderate flares", "severe flares"], inplace=True)

# Display the first few rows to see the new columns
df.head()

### 3.2. Data Preprocessing

The raw sunspot data requires preprocessing to prepare it for machine learning algorithms. This preprocessing phase is critical for solar flare prediction because the success of our model depends heavily on how well the categorical and ordinal features are transformed into numerical representations that preserve their inherent relationships and physical meaning.

**Key Preprocessing Steps:**

- **Ordinal Encoding:** `largest spot size` and `spot distribution` are converted to numerical scales (1-6 and 1-4 respectively) that preserve their inherent ordering from least to most complex/energetic.

- **Binary Feature Standardization:** Five features are identified as binary and converted to standard 0/1 encoding:
  - `historically-complex` and `became complex on this pass`: 0 = "no" (not complex), 1 = "yes" (complex)
  - `activity`: 0 = "decay", 1 = "no change"
  - `area` and `area of largest spot`: 0 = smaller size/area, 1 = larger size/area
  
  This follows ML best practices and ensures intuitive interpretation where higher values indicate greater complexity or size.

- **One-Hot Encoding:** The `modified Zurich class` feature is transformed using one-hot encoding to prevent the algorithm from inferring incorrect ordinal relationships between the different sunspot classifications.

This preprocessing approach optimizes compatibility with machine learning algorithms. Ordinal relationships are preserved and binary features are clearly interpretable.


In [ ]:
# Define the order for each ordinal feature
largest_spot_size_order = {"X": 1, "R": 2, "S": 3, "A": 4, "H": 5, "K": 6}
spot_distribution_order = {"X": 1, "O": 2, "I": 3, "C": 4}

# Map the string categories to their ordinal values
df["largest spot size"] = df["largest spot size"].map(largest_spot_size_order)
df["spot distribution"] = df["spot distribution"].map(spot_distribution_order)

# Convert all binary categorical features to standard 0/1 encoding
df["historically-complex"] = (df["historically-complex"] == 1).astype(int)  # 0=no, 1=yes
df["became complex on this pass"] = (df["became complex on this pass"] == 1).astype(int)  # 0=no, 1=yes
df["activity"] = (df["activity"] == 2).astype(int)  # 0=decay, 1=no change
df["area"] = (df["area"] == 2).astype(int)  # 0=small, 1=large
df["area of largest spot"] = (df["area of largest spot"] == 2).astype(int)  # 0=<=5, 1=>5

# One-hot encode the modified Zurich class feature
categorical_cols = ["modified Zurich class"]
df_encoded = pd.get_dummies(df, columns=["modified Zurich class"])

print("Dataset shape:", df_encoded.shape)
print("\nBinary feature distributions:")
binary_features = ["historically-complex", "became complex on this pass", "activity", "area", "area of largest spot"]
for feature in binary_features:
    print(f"{feature}: {df_encoded[feature].value_counts().to_dict()}")

df_encoded.head()

# For SVM only - add after the encoding steps
# scaler = sk.preprocessing.StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

### 3.3. Feature Correlation Analysis

To better understand the relationships between input features and the target variable, we visualize the correlation matrix of the fully encoded dataset. This analysis reveals critical insights about feature predictiveness.

**Key Insights from Correlation Analysis:**

**Target Variable Relationships (`flare_class`):**
The correlation heatmap reveals specific patterns in how sunspot characteristics relate to flare production:

- **Strong Positive Predictors:**
  - `modified Zurich class` `D`, `E`, and `F`: Complex magnetic configurations increase flare probability
  - `spot distribution` and `largest spot size`: Classifications similar to `modified Zurich class` related to compactness and size, respectively
  - `area`: Larger sunspot groups are more likely to produce higher-class flares
  - `previous 24 hour flare activity`: Recent flare history predicts continued activity

- **Negative Correlations:**
  - `modified Zurich class` `B`, `C`, and `H`: Simple or decayed sunspot regions have reduced flare potential
  - `historically-complex` and `became complex on this pass`: Complexity factors have negative correlations, which requires deeper understanding of the dataset and solar physics. The `historically-complex` feature shows negative correlation because most historically-complex regions decay to the H-class, which is highly represented in the dataset. The `became complex on this pass` feature may indicate that it takes longer than 24 hours after a region has become complex for it to produce solar flares.

**Multicollinearity Assessment:**
Most feature correlations remain within acceptable ranges (|r| < 0.7), indicating that our feature set provides complementary rather than redundant information. The strongest correlations are between logically related features, which is expected and manageable.

**Physical Interpretation:**
These findings align with solar physics understanding that:
1. **Flare productivity peaks during active development phases** rather than historical complexity periods
2. **Magnetic complexity evolution is temporal** - regions cycle through phases of growth, peak activity, and decay
3. **Current morphological characteristics** (spot size, distribution, area) are more predictive of immediate flare potential than historical or developmental complexity flags

**Implications for Model Development:**
This analysis suggests our models should prioritize:
- Current physical characteristics over historical complexity indicators
- Active Zurich classes (D, E, F) rather than complexity flags
- Recent activity patterns and morphological features as primary predictors

This counterintuitive pattern actually strengthens confidence in our dataset's physical validity, as it captures the realistic temporal evolution of solar active regions from development through decay phases.

In [ ]:
# Compute the correlation matrix
corr_matrix = df_encoded.corr()

# Set up the figure size
plt.figure(figsize=(18, 14))

# Draw the heatmap
sns.heatmap(
    corr_matrix,
    annot=False,
    cmap="coolwarm",
    center=0,
    linewidths=0.5,
    cbar_kws={"shrink": 0.9},
    xticklabels=True,
    yticklabels=True,
)

plt.title("Correlation Heat Map of Encoded Features and Target", fontsize=24)
plt.xticks(fontsize=16, rotation=45, ha="right")
plt.yticks(fontsize=16)
plt.tight_layout()
plt.show()

# Target correlations
target_corr = df_encoded.corr()['flare_class'].sort_values(ascending=False)
print("Target correlations")
print("Top positive predictors:")
print(target_corr.head(10)[1:-1])
print("\nTop negative predictors:")
print(target_corr.tail(5)[::-1])

### 3.4. Data Splitting

The final step in data preparation is to split the dataset into training and testing sets. This separation is crucial for evaluating the model's performance on unseen data and preventing overfitting.

**Key Considerations for Solar Flare Prediction:**

Given the severe class imbalance identified in our exploratory analysis, stratified sampling should be employed to ensure both training and testing sets maintain the same proportion of flare classes. This is particularly critical for the rare but dangerous X-class flares, which represent only 1% of the dataset.

**Split Strategy:**

- **Training Set (80%):** Used to train the machine learning model
- **Testing Set (20%):** Reserved for final model evaluation
- **Stratified Sampling:** Maintains class distribution across both sets
- **Random State:** Fixed for reproducibility of results

The resulting datasets will be used to train and evaluate our classification model, with higher attention to the model's ability to detect M and X class flares.


In [ ]:
# Split the data into features (X) and target (y)
X = df_encoded.drop(columns=["flare_class"])
y = df_encoded["flare_class"]

# Split the data into training and testing sets
# Using stratified sampling to maintain class distribution in both sets
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Display the shapes of the resulting datasets
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)
print("\nClass distribution in training set:")
print(y_train.value_counts().sort_index())
print("\nClass distribution in testing set:")
print(y_test.value_counts().sort_index())

## 4. Modeling